In [12]:
from neo4j import GraphDatabase
import csv

In [16]:
uri = "bolt://10.2.42.255:7687"  # Change this to your Neo4j instance URI
username = "neo4j"             # Replace with your username
password = "pass12345"          # Replace with your password
driver = GraphDatabase.driver(uri, auth=(username, password))

In [33]:
from neo4j import GraphDatabase

# Define allowed labels to prevent injection attacks
ALLOWED_LABELS = {'PERSON', 'GPE', 'ORG', 'POL', 'DIR', 'IAS', 'COMP'}

def get_label(entity_type):
    entity_type = entity_type.upper()
    return entity_type if entity_type in ALLOWED_LABELS else 'ENTITY'

def create_nodes_from_csv(csv_file):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        with open(csv_file, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                entity_name = row['Entity'].strip()
                entity_type = row['Type'].strip()
                label = get_label(entity_type)

                # Merge Entity node
                query_node = f"MERGE (e:{label} {{name: $name}})"
                session.run(query_node, name=entity_name)
    driver.close()

# Path to your entities CSV file
entities_csv_file = 'entities.csv'

# Create nodes in Neo4j
create_nodes_from_csv(entities_csv_file)


ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)

In [17]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record["n"]["name"] for record in result]


In [18]:
query = "MATCH (n) RETURN n LIMIT 10"
nodes = run_query(query)
print(nodes)

[]


In [9]:
import csv

csv_file_path = 'cooccurrences.csv'

with open(csv_file_path, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]

In [10]:
def add_weighted_relationship(tx, entity1, entity2, weight):
    query = """
    MERGE (e1:Entity {name: $entity1})
    MERGE (e2:Entity {name: $entity2})
    MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
    SET r.weight = $weight
    """
    tx.run(query, entity1=entity1, entity2=entity2, weight=weight)


In [11]:
with driver.session() as session:
    for row in data:
        entity1 = row['entity1']
        entity2 = row['entity2']
        weight = int(row['weight'])
        
        session.write_transaction(add_weighted_relationship, entity1, entity2, weight)


/var/folders/b3/b3lpk8w51qj10znm_7bkftgr0000gn/T/ipykernel_2077/2257033392.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_weighted_relationship, entity1, entity2, weight)


KeyboardInterrupt: 

In [15]:
driver.close()


In [ ]:
# CHECK THE NODE RELATIONSHIPS ALONG WITH WEIGHTS

# MATCH (e1:Entity)-[r:CO_OCCURS_WITH]->(e2:Entity)
# RETURN e1, r, e2
# LIMIT 100

In [1]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "mediagraph"
driver = GraphDatabase.driver(uri, auth=(username, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

node_count_query = "MATCH (n:Entity) RETURN COUNT(DISTINCT n) as nodeCount"
edge_count_query = "MATCH ()-[r:CO_OCCURS_WITH]-() RETURN COUNT(DISTINCT r) as edgeCount"

node_count_result = run_query(node_count_query)
edge_count_result = run_query(edge_count_query)

if node_count_result and edge_count_result:
    node_count = node_count_result[0][0]
    edge_count = edge_count_result[0][0]

    if node_count > 1:
        density = 2 * edge_count / (node_count * (node_count - 1))
        print(f"Graph Density: {density:.4f}")
    else:
        print("Insufficient nodes to calculate density.")
else:
    print("Failed to fetch data from the database.")

driver.close()


Graph Density: 0.0017


In [2]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "neo4j://localhost:7687"
username = "neo4j"
password = "mediagraph"
driver = GraphDatabase.driver(uri, auth=(username, password))

def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return [record for record in result]

# Use GDS to create a graph projection
create_graph_query = """
CALL gds.graph.create(
    'entitiesGraph',
    'Entity',
    {
        CO_OCCURS_WITH: {
            type: 'CO_OCCURS_WITH',
            orientation: 'UNDIRECTED'
        }
    }
)
"""

# Calculate betweenness centrality
betweenness_query = """
CALL gds.betweenness.stream('entitiesGraph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS node, score
ORDER BY score DESC
LIMIT 10
"""

try:
    # Create the graph projection
    run_query(create_graph_query)
    print("Graph projection created.")

    # Execute betweenness centrality calculation
    betweenness_results = run_query(betweenness_query)
    for result in betweenness_results:
        print(f"Node: {result['node']}, Betweenness Centrality: {result['score']:.4f}")
finally:
    # Drop the graph projection after computation
    run_query("CALL gds.graph.drop('entitiesGraph')")
    print("Graph projection dropped.")

    # Close driver connection
    driver.close()


ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `entitiesGraph` does not exist on database `neo4j`. It might exist on another database.}